# 1. Setup

1.1 Importing Libraries

In [ ]:
import cv2
import os
import random
import numpy as np 
from PIL import Image
from matplotlib import pyplot as plt
import uuid

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

1.2 Set GPU Growth

In [ ]:
# grabs all GPUs
gpus = tf.config.experimental.list_physical_devices("GPU")
# sets memory growth for each GPU
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

1.3 Create Folder Structures

In [ ]:
POS_PATH = os.path.join("data", "positive2")
NEG_PATH = os.path.join("data", "negative")
ANC_PATH = os.path.join("data", "anchor2")

In [ ]:
# make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# 2. Collect Positives and Anchors

2.1 Untar Labelled Faces in the Wild Dataset

In [ ]:
# http://vis-www.cs.umass.edu/lfw/

In [ ]:
# uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [ ]:
# move LFW Images to the following repository data/negative
for directory in os.listdir("lfw"):
    for file in os.listdir(os.path.join("lfw", directory)):
        EX_PATH = os.path.join("lfw", directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

2.2 Collect Postitive and Anchor Images

In [ ]:
# import uuid (universally unique identifiers) library to generate unique image name 
import uuid

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 250)
cap.set(4, 250)

while True:
    ret, frame = cap.read() # reads each frame one by one
    
    # create positives
    if cv2.waitKey(1) & 0xFF == ord("p"):
        imname = os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imname, frame)
    # create anchors
    if cv2.waitKey(1) & 0xFF == ord("a"):
        imname = os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imname, frame)



    cv2.imshow("frame", frame) # shows image 
    if cv2.waitKey(20) & 0xFF == 27: # allows keyboard press to close window with the escape key
        break

# when finished, release the capture
cap.release() 
cv2.destroyAllWindows()

In [ ]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250pxqqqqqq
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord("a"):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord("p"):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow("Image Collection", frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord("q"):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

# 3. Load and Pre-process Images

3.1 Get Image Directories

In [ ]:
# loads first 300 images in each of the directories
anchor = tf.data.Dataset.list_files(ANC_PATH+"/*.jpg").take(300)
positive = tf.data.Dataset.list_files(POS_PATH+"/*.jpg").take(300) 
negative = tf.data.Dataset.list_files(NEG_PATH+"/*.jpg").take(300)

3.2 Preprocessing - Scale and Resize

In [ ]:
def preprocess(file_path):
    # reading img
    byte_img = tf.io.read_file(file_path)
    # using tf decode image to load it in
    img = tf.io.decode_jpeg(byte_img)
    # resizes img
    img = tf.image.resize(img, (100,100))
    # scales image to be between 0 and 1
    img = img / 255.0
    return img

3.3 Create Labelled Dataset

In [ ]:
# combines the anchor and positve/negative image. Adds 1.0/0.0 depending if same face
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

3.4 Build Train and Test Partition

In [ ]:
# puts input_img, validation_img and label into a list
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

In [ ]:
#### Build dataLoader Pipeline ####
# maps our data
data = data.map(preprocess_twin)
# caching our images so we can access them faster
data = data.cache()
# shuffles all our data,
data = data.shuffle(buffer_size=1024)

In [ ]:
#### Training Partition ####
# takes 70% of images for training data
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
# starts preprocessing the next set of images so that we don"t bottle neck our next set images
train_data = train_data.prefetch(8)

In [ ]:
#### Testing Partition ####
# skips first 70% of images
test_data = data.skip(round(len(data)*.7))
# then takes 30% of data left 
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# 4. Model Engineering

4.1 Build Embedding Layer

In [ ]:
inp = Input(shape=(100,100,3), name="input_image")

In [ ]:
inp = Input(shape=(100,100,3), name="input_image")

c1 = Conv2D(64, (10,10), activation="relu")(inp)
m1 = MaxPooling2D(64, (2,2), padding="same")(c1)

c2 = Conv2D(128, (7,7), activation="relu")(m1)
m2 = MaxPooling2D(64, (2,2), padding="same")(c2)

c3 = Conv2D(128, (4,4), activation="relu")(m2)
m3 = MaxPooling2D(64, (2,2), padding="same")(c3)

c4 = Conv2D(256, (4,4), activation="relu")(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation="sigmoid")(f1)

mod = Model(inputs=[inp], outputs=[d1], name="embedding")
mod.summary()

In [ ]:
def make_embedding():
    inp = Input(shape=(100,100,3), name="input_image")

    #### First block ####
    c1 = Conv2D(64, (10,10), activation="relu")(inp)
    m1 = MaxPooling2D(64, (2,2), padding="same")(c1)

    #### Second Block ####
    c2 = Conv2D(128, (7,7), activation="relu")(m1)
    m2 = MaxPooling2D(64, (2,2), padding="same")(c2)

    #### Third Block ####
    c3 = Conv2D(128, (4,4), activation="relu")(m2)
    m3 = MaxPooling2D(64, (2,2), padding="same")(c3)

    #### Final Embedding Block ####
    c4 = Conv2D(256, (4,4), activation="relu")(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation="sigmoid")(f1)
    return Model(inputs=[inp], outputs=[d1], name="embedding")

In [ ]:
embedding = make_embedding()
embedding.summary()

Distance layer

In [ ]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

Make model

In [ ]:
def make_siamese_model():
    input_image = Input(name="input_img", shape=(100,100,3))
    validation_image = Input(name="validation_img", shape=(100,100,3))

    
    
    siamese_layer = L1Dist()
    siamese_layer._name = "distance"
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    classifier = Dense(1, activation="sigmoid")(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name="SiameseNetwork")

In [ ]:
siamese_model = make_siamese_model()
siamese_model.summary()

# 5. Training

5.1 Setup Loss and Optimiser

In [ ]:
# the loss will be used later to be able to calculate our loss (1 or 0)
binary_loss = tf.losses.BinaryCrossentropy()

In [ ]:
# improves speed and performance
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

5.2 Establish Checkpoints

In [ ]:
# defined our checkpoint dir
checkpoint_dir = "./training_checkpoints"
# ensures that all our checkpoints have the prefix of ckpt
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# saves our the model and optimiser at the time we run the checkpoint class
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

5.3 Build Train Step Function

In [ ]:
@tf.function # compiles our function into a callable TensorFlow graph
def train_step(batch):
    # allows us to capture our gradient from the model, records the operations for automatic differentiation
    with tf.GradientTape() as tape:
        x = batch[:2] # get anchor and positive/negative images
        y = batch[2] # takes the label

        # passes our data through the siamese model to make a prediction
        y_pred = siamese_model(x, training=True)
        # calculates the loss
        loss = binary_loss(y, y_pred)
    # calculates all of the gradients in respect to our loss for all of our trainable variables   
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    # calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))

    return loss



5.4 Build Training Loop

In [ ]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print(f"\n Epoch {epoch}/{EPOCHS}")
        progress_bar = tf.keras.utils.Progbar(len(data))

    for idx, batch in enumerate(data):
        train_step(batch)
        progress_bar.update(idx+1)

    if epoch % 10 == 0: 
        checkpoint.save(file_prefix=checkpoint_prefix)

5.5 Train Model

In [ ]:
EPOCHS = 50 # num times we will run through the training data
train(train_data, EPOCHS)

In [ ]:
siamese_model.save("siamese_model.h5")